In [1]:
!pip install keybert

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 47.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.8 MB/s eta 0:00:00
  Created wheel for keybert: filename=keybert-0.8.3-py3-none-any.whl size=39124 sha256=e10dce7b73be99a81d41c4eda01abb7fc59b62ebf5997a7f739e91afe404111e
  Stored in directory: /root/.cache/pip/wheels/70/88/07/1a3bc11fd1dd5f89924a02dcbca89a3015e25e8faa31f904dc
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl 

In [2]:
import numpy as np
import pandas as pd
import re
from keybert import KeyBERT

In [3]:
df = pd.read_csv("main_dataset.csv", dtype={'HS_Code': object})

In [4]:
df.drop_duplicates(inplace=True)

In [5]:
df.head(10)

,HS_Code,Description
0,500100,DISCONNECTORS SUITABLE FOR S3CD 362/1250
8,500100,FREIGHT PREPAID CARGO IS STOWED IN A REFRIGERA...
23,500100,"705 CTN & 12,488 DZ PRS 2700 DZ PRS 70 COTTON ..."
25,500100,"OTHER, SUITABLE FOR FURNIT UREOTHER, SUITABLE ..."
26,500100,TUNNEL WARMER SUITABLE TUNNEL WARMER SUITABLE ...
30,500100,"MAIN REEL, REELING EHTF HEAD, AND STANDARD RE"
44,520819,IKEA HOME FURNISHING PRODUCTS TARIFF NUMBER: 9...
45,520819,IKEA HOME FURNISHING PRODUCTS TARIFF NUMBER: 9...
46,520819,FROZEN SWAI FILLETS FDA RE GISTRATION NUMBER :...
47,520819,FROZEN SWAI FILLETS FDA RE GISTRATION NUMBER :...


In [6]:
df['Description'] = df['Description'].str.lower()
df['Description'] = df['Description'].str.replace('\W', ' ')
df['Description'] = df['Description'].apply(lambda x: x.split('hs code', 1)[0] if len(x.split('hs code', 1)[0]) > 0 else x.split('hs code', 1)[1])
df['Description'] = df['Description'].apply(lambda x: x.split('hts code', 1)[0] if len(x.split('hts code')[0]) > 0 else x.split('hts code', 1)[1])
df['Description'] = df['Description'].apply(lambda x: x.split('h t s code', 1)[0] if len(x.split('h t s code', 1)[0]) > 0 else x.split('h t s code', 1)[1])
df['Description'] = df['Description'].apply(lambda x: x.split('hscode', 1)[0] if len(x.split('hscode', 1)[0]) > 0 else x.split('hscode', 1)[1])
df['Description'] = df['Description'].apply(lambda x: x.split('h s code', 1)[0] if len(x.split('h s code', 1)[0]) > 0 else x.split('h s code', 1)[1])
df['Description'] = df['Description'].apply(lambda x: x.split('htscode', 1)[0] if len(x.split('htscode', 1)[0]) > 0 else x.split('htscode', 1)[1])
df['Description'] = df['Description'].apply(lambda x: re.sub(r'\d{6}|\d{10}','', x))
df['Description'] = df['Description'].apply(lambda x: re.sub(r'\d{4}\ \d{2}','', x))


In [7]:
df.head(10)

,HS_Code,Description
0,500100,disconnectors suitable for s3cd 362 1250
8,500100,freight prepaid cargo is stowed in a refrigera...
23,500100,705 ctn 12 488 dz prs 2700 dz prs 70 cotton ...
25,500100,other suitable for furnit ureother suitable ...
26,500100,tunnel warmer suitable tunnel warmer suitable ...
30,500100,main reel reeling ehtf head and standard re
44,520819,ikea home furnishing products tariff number 9...
45,520819,ikea home furnishing products tariff number 9...
46,520819,frozen swai fillets fda re gistration number ...
47,520819,frozen swai fillets fda re gistration number ...


In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test, y_train, y_test = train_test_split(df, df['HS_Code'], test_size=0.1)

In [9]:
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')

In [40]:
def extract_keywords(keyword_score_list):
    phrase_list = []
    if len(keyword_score_list) > 0:
        for keyword in keyword_score_list:
            if keyword[1] >= 0.1:
                phrase_list.append(keyword[0])
        phrase_non_unique = " ".join(phrase_list)
        unique_words = set(phrase_non_unique.split(' '))
        phrase = " ".join(unique_words)

        return phrase
    else:
        return ""

In [14]:
from tqdm import tqdm

unique_code = list(set(df_train['HS_Code'].tolist()))
code_keyword_map = {}
for code in tqdm(unique_code):
    concat_description = " ".join(df_train[df_train['HS_Code'] == code]['Description'].tolist())
    keywords_scores = kw_model.extract_keywords(concat_description,keyphrase_ngram_range=(1, 3), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)
    extracted_keywords = extract_keywords(keywords_scores)
    code_keyword_map[code] = extracted_keywords

100%|██████████| 519/519 [2:39:41<00:00, 18.46s/it]


In [15]:
import json
# Saving the dictionary to a JSON file
with open('code_keyword_map.json', 'w') as file:
    json.dump(code_keyword_map, file)

print("Saved code_keyword_map to code_keyword_map.json")

Saved code_keyword_map to code_keyword_map.json


In [ ]:
#df_train.to_csv("keyword_extract.csv")

In [16]:
descriptions = df_test['Description'].tolist()

In [17]:
keywords = kw_model.extract_keywords(descriptions,keyphrase_ngram_range=(1, 3), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)

In [18]:
df_test['Keyword_with_Scores'] = keywords

In [19]:
df_test.head(10)

,HS_Code,Description,Keyword_with_Scores
1855,521159,furnishing fabrics po no invoice no exgms 22...,"[(date 10, 0.33), (18, 0.3626), (furnishing fa..."
38169,550330,dolan fas type 25x 2 2 50mm glsf 89117,"[(type, 0.2997), (dolan, 0.4022), (89117, 0.50..."
34687,540810,ladies viscose woven dress,"[(woven, 0.5698), (viscose, 0.58), (ladies, 0...."
28743,600622,men s 100 cotton knit tops,"[(tops, 0.2086), (men, 0.3363), (cotton knit t..."
26573,560790,slac linen ficelle 5 parcels,"[(slac, 0.5432), (parcels, 0.5519), (ficelle, ..."
21535,551443,polyester spandex satin fabric,"[(fabric, 0.5617), (spandex, 0.5628), (satin, ..."
4511,520531,yarn,[]
33634,611699,silk gloves,[]
20955,551441,mens flc pullover slac 705 boxes with mens flc...,"[(15 polyester sc, 0.4383), (50 cotton 50, 0.4..."
6181,531090,hemp woven mens shirt,"[(woven, 0.725), (shirt, 0.7341), (hemp, 0.760..."


In [41]:
df_test['Keywords'] = df_test['Keyword_with_Scores'].apply(extract_keywords)

In [42]:
df_test.head(10)

,HS_Code,Description,Keyword_with_Scores,Keywords,Prediction
1855,521159,furnishing fabrics po no invoice no exgms 22...,"[(date 10, 0.33), (18, 0.3626), (furnishing fa...",date furnishing exgms 18 fabrics 22 10 invoice,551692
38169,550330,dolan fas type 25x 2 2 50mm glsf 89117,"[(type, 0.2997), (dolan, 0.4022), (89117, 0.50...",89117 25x 50mm glsf type dolan,550330
34687,540810,ladies viscose woven dress,"[(woven, 0.5698), (viscose, 0.58), (ladies, 0....",woven dress ladies viscose,610341
28743,600622,men s 100 cotton knit tops,"[(tops, 0.2086), (men, 0.3363), (cotton knit t...",cotton men knit tops 100,551441
26573,560790,slac linen ficelle 5 parcels,"[(slac, 0.5432), (parcels, 0.5519), (ficelle, ...",ficelle parcels linen slac,521029
21535,551443,polyester spandex satin fabric,"[(fabric, 0.5617), (spandex, 0.5628), (satin, ...",fabric spandex polyester satin,551423
4511,520531,yarn,[],,550700
33634,611699,silk gloves,[],,611691
20955,551441,mens flc pullover slac 705 boxes with mens flc...,"[(15 polyester sc, 0.4383), (50 cotton 50, 0.4...",705 polyester crew cotton boxes mens 15 50 10 ...,540252
6181,531090,hemp woven mens shirt,"[(woven, 0.725), (shirt, 0.7341), (hemp, 0.760...",mens woven shirt hemp,531090


In [43]:
df_test['Keywords'] = np.where(df_test['Keyword_with_Scores'].str.len() == 0, df_test['Description'].str.lower(), df_test['Keywords'])

In [44]:
df_test.head(10)

,HS_Code,Description,Keyword_with_Scores,Keywords,Prediction
1855,521159,furnishing fabrics po no invoice no exgms 22...,"[(date 10, 0.33), (18, 0.3626), (furnishing fa...",date furnishing exgms 18 fabrics 22 10 invoice,551692
38169,550330,dolan fas type 25x 2 2 50mm glsf 89117,"[(type, 0.2997), (dolan, 0.4022), (89117, 0.50...",89117 25x 50mm glsf type dolan,550330
34687,540810,ladies viscose woven dress,"[(woven, 0.5698), (viscose, 0.58), (ladies, 0....",woven dress ladies viscose,610341
28743,600622,men s 100 cotton knit tops,"[(tops, 0.2086), (men, 0.3363), (cotton knit t...",cotton men knit tops 100,551441
26573,560790,slac linen ficelle 5 parcels,"[(slac, 0.5432), (parcels, 0.5519), (ficelle, ...",ficelle parcels linen slac,521029
21535,551443,polyester spandex satin fabric,"[(fabric, 0.5617), (spandex, 0.5628), (satin, ...",fabric spandex polyester satin,551423
4511,520531,yarn,[],yarn,550700
33634,611699,silk gloves,[],silk gloves,611691
20955,551441,mens flc pullover slac 705 boxes with mens flc...,"[(15 polyester sc, 0.4383), (50 cotton 50, 0.4...",705 polyester crew cotton boxes mens 15 50 10 ...,540252
6181,531090,hemp woven mens shirt,"[(woven, 0.725), (shirt, 0.7341), (hemp, 0.760...",mens woven shirt hemp,531090


In [45]:
def jaccard_similarity(str1, str2):
    set1 = set(str1.split(' '))
    set2 = set(str2.split(' '))
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

In [83]:
def get_best_match_code(prod_keywords, code_keyword_map):

    best_similarity = 0
    best_code = "None"

    for hs_code, code_keywords in code_keyword_map.items():
        similarity = jaccard_similarity(prod_keywords, code_keywords)
        if similarity > best_similarity:
            best_similarity = similarity
            best_code = hs_code

    return best_code

In [84]:
df_test['Prediction'] = df_test['Keywords'].apply(lambda x: get_best_match_code(x, code_keyword_map))

In [85]:
df_test.head(10)

,HS_Code,Description,Keyword_with_Scores,Keywords,Prediction,Prediction_ST
1855,521159,furnishing fabrics po no invoice no exgms 22...,"[(date 10, 0.33), (18, 0.3626), (furnishing fa...",date furnishing exgms 18 fabrics 22 10 invoice,551692,520919
38169,550330,dolan fas type 25x 2 2 50mm glsf 89117,"[(type, 0.2997), (dolan, 0.4022), (89117, 0.50...",89117 25x 50mm glsf type dolan,550330,550330
34687,540810,ladies viscose woven dress,"[(woven, 0.5698), (viscose, 0.58), (ladies, 0....",woven dress ladies viscose,610341,511300
28743,600622,men s 100 cotton knit tops,"[(tops, 0.2086), (men, 0.3363), (cotton knit t...",cotton men knit tops 100,551441,551441
26573,560790,slac linen ficelle 5 parcels,"[(slac, 0.5432), (parcels, 0.5519), (ficelle, ...",ficelle parcels linen slac,521029,540832
21535,551443,polyester spandex satin fabric,"[(fabric, 0.5617), (spandex, 0.5628), (satin, ...",fabric spandex polyester satin,551423,540832
4511,520531,yarn,[],yarn,550700,551621
33634,611699,silk gloves,[],silk gloves,611691,540832
20955,551441,mens flc pullover slac 705 boxes with mens flc...,"[(15 polyester sc, 0.4383), (50 cotton 50, 0.4...",705 polyester crew cotton boxes mens 15 50 10 ...,540252,540252
6181,531090,hemp woven mens shirt,"[(woven, 0.725), (shirt, 0.7341), (hemp, 0.760...",mens woven shirt hemp,531090,611219


In [96]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

In [93]:
print(accuracy_score(df_test['HS_Code'], df_test['Prediction'])*100)

17.94871794871795


In [102]:
print(precision_score(df_test['HS_Code'], df_test['Prediction'], average='weighted', zero_division=0))

0.30709233760307875


In [90]:
print(f1_score(df_test['HS_Code'], df_test['Prediction'], average='weighted'))

0.1867356902071456


# Sentence Transformer

In [51]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

In [56]:
desc_sentence_embeddings = model.encode(df_test['Keywords'].tolist())

code_keyword_embed = {}
for hs_code, code_keywords in code_keyword_map.items():
        embed_keywords = model.encode(code_keywords)
        code_keyword_embed[hs_code] = embed_keywords

In [57]:
from sklearn.metrics.pairwise import cosine_similarity

def get_best_code_st(desc_sentence_embeddings, code_keyword_embed):
    best_codes = []

    for phrase_embed in desc_sentence_embeddings:
        best_similarity = 0
        best_code = None

        for hs_code, code_embed in code_keyword_embed.items():
            similarity = cosine_similarity([phrase_embed],[code_embed])[0]
            if similarity > best_similarity:
                best_similarity = similarity
                best_code = hs_code

        best_codes.append(best_code)

    return best_codes


In [58]:
df_test['Prediction_ST'] = get_best_code_st(desc_sentence_embeddings, code_keyword_embed)

In [59]:
df_test.head(10)

,HS_Code,Description,Keyword_with_Scores,Keywords,Prediction,Prediction_ST
1855,521159,furnishing fabrics po no invoice no exgms 22...,"[(date 10, 0.33), (18, 0.3626), (furnishing fa...",date furnishing exgms 18 fabrics 22 10 invoice,551692,520919
38169,550330,dolan fas type 25x 2 2 50mm glsf 89117,"[(type, 0.2997), (dolan, 0.4022), (89117, 0.50...",89117 25x 50mm glsf type dolan,550330,550330
34687,540810,ladies viscose woven dress,"[(woven, 0.5698), (viscose, 0.58), (ladies, 0....",woven dress ladies viscose,610341,511300
28743,600622,men s 100 cotton knit tops,"[(tops, 0.2086), (men, 0.3363), (cotton knit t...",cotton men knit tops 100,551441,551441
26573,560790,slac linen ficelle 5 parcels,"[(slac, 0.5432), (parcels, 0.5519), (ficelle, ...",ficelle parcels linen slac,521029,540832
21535,551443,polyester spandex satin fabric,"[(fabric, 0.5617), (spandex, 0.5628), (satin, ...",fabric spandex polyester satin,551423,540832
4511,520531,yarn,[],yarn,550700,551621
33634,611699,silk gloves,[],silk gloves,611691,540832
20955,551441,mens flc pullover slac 705 boxes with mens flc...,"[(15 polyester sc, 0.4383), (50 cotton 50, 0.4...",705 polyester crew cotton boxes mens 15 50 10 ...,540252,540252
6181,531090,hemp woven mens shirt,"[(woven, 0.725), (shirt, 0.7341), (hemp, 0.760...",mens woven shirt hemp,531090,611219


In [60]:
print(accuracy(df_test['HS_Code'], df_test['Prediction_ST'])*100)

7.785547785547786


# Keyword Analysis

In [61]:
def calc_precision(extr_keywords, ref_keywords):

    # Out of the extracted keywords, checks how many are in the reference
    set1 = set(extr_keywords.split(' '))
    set2 = set(ref_keywords.split(' '))
    intersection = set1.intersection(set2)

    return len(intersection) / len(set1)

In [62]:
def avg_precision(ex_kw_list, ref_kw_list):
    prec_sum = 0
    for ex_kws, ref_kws in zip(ex_kw_list, ref_kw_list):
        prec_sum += calc_precision(ex_kws, ref_kws)

    return round(prec_sum/len(ex_kw_list),3)


In [63]:
ref_df = pd.read_csv("Reference_keywords.csv")
ref_df['Description'] = ref_df['Description'].str.lower()
ref_df['Description'] = ref_df['Description'].str.replace('\W', ' ')
ref_df['Reference Keywords'] = ref_df['Reference Keywords'].str.replace('\W', ' ')
ref_df['Reference Keywords'] = ref_df['Reference Keywords'].str.lower()
ref_df.head()

,HS Code,Description,Reference Keywords
0,600642,knitted dyed fleece unisex premium crew neck k...,knitted dyed fleece unisex premium crew neck z...
1,600641,carbonx active knit undwrwear tops with hoods ...,carbonx active knit underwear twill woven fabric
2,600641,men s cotton knit polo interlockgarments style...,men s cotton knit polo interlock garments
3,600634,ladies 52percent polyester recycled 42percent ...,ladies 52percent polyester recycled 42percent ...
4,600633,119 cartons 975 05 kgm 8 418 mtq boy 60 cotton...,cotton polyester knit pullover double dye pull...


In [64]:
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')

In [65]:
keywords = kw_model.extract_keywords(ref_df['Description'].tolist(),keyphrase_ngram_range=(1, 3), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)

In [66]:
ref_df['Keywords'] = keywords

In [67]:
ref_df['Extracted Keywords'] = ref_df['Keywords'].apply(extract_keywords)

In [68]:
ref_df.head()

,HS Code,Description,Reference Keywords,Keywords,Extracted Keywords
0,600642,knitted dyed fleece unisex premium crew neck k...,knitted dyed fleece unisex premium crew neck z...,"[(crew neck knitted, 0.4621), (premium crew ne...",fleece crew dyed zipper white premium hoodie k...
1,600641,carbonx active knit undwrwear tops with hoods ...,carbonx active knit underwear twill woven fabric,"[(carbonx active knit, 0.5101), (underwear top...",floves bottoms underwear knit carbonx string a...
2,600641,men s cotton knit polo interlockgarments style...,men s cotton knit polo interlock garments,"[(78j3011 po 4300015906, 0.4618), (78j3005 po ...",4300015906 cotton knit 4300001943 polo 100 men...
3,600634,ladies 52percent polyester recycled 42percent ...,ladies 52percent polyester recycled 42percent ...,"[(3percent nylon, 0.3735), (recycled 42percent...",42percent nylon polyester recycled ladies 3per...
4,600633,119 cartons 975 05 kgm 8 418 mtq boy 60 cotton...,cotton polyester knit pullover double dye pull...,"[(583085 style 3024175, 0.3459), (waffle doubl...",waffle 583085 double polyester ctns cartons 20...


In [69]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('bert-base-nli-mean-tokens')

extr_sentence_embeddings = model.encode(ref_df['Extracted Keywords'].tolist())
ref_sentence_embeddings = model.encode(ref_df['Reference Keywords'].tolist())

In [70]:
def avg_cosine_sim(extr_sentence_embeddings, ref_sentence_embeddings):
    # Finds the average cosine similarity between the extracted keywords and the reference keywords
    cos_sim = 0
    for ex_kws, ref_kws in zip(extr_sentence_embeddings, ref_sentence_embeddings):
        cos_sim += cosine_similarity([ex_kws],[ref_kws])[0][0]

    return round(cos_sim/len(extr_sentence_embeddings),3)

In [71]:
avg_precision(ref_df['Extracted Keywords'].tolist(), ref_df['Reference Keywords'].tolist())

0.525

In [72]:
avg_cosine_sim(extr_sentence_embeddings, ref_sentence_embeddings)

0.767